# R Code to Python Conversion: Geocoding & Spatial Analysis

이 노트북은 `지오코딩python변경.qmd` 파일에 있던 R 코드를 Python으로 변환한 내용입니다.
주요 단계:
1. 데이터 수집 (공공데이터포털)
2. 데이터 전처리
3. 지오코딩 (Kakao API)
4. 공간 데이터 및 그리드 생성
5. 공간 분석 (KDE) 및 시각화

In [5]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import os
from datetime import datetime
import time
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, box
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import folium
from folium import plugins
import matplotlib.colors as mcolors

# 시각화 한글 폰트 설정 (Windows 기준)
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
if os.path.exists(font_path):
    font = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=font)
plt.rcParams['axes.unicode_minus'] = False

def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

## 1. 데이터 수집 (Data Collection)
국토교통부 아파트매매 실거래자료 API를 사용하여 데이터를 수집합니다.

In [6]:
def collect_data():
    print(">>> 데이터 수집 시작")
    
    # 설정값
    lawd_cd = 41390  # 법정동코드
    service_key = 'yQ3x5TpauR4TZTv3VVe7/GtT5ooGmknYKee5L9a5oOM0/oOHiKURlQ0FamByEQFWGM0NYNg204bG9mXY17T6HQ=='
    
    # 날짜 리스트 생성 (2006-01 ~ 2025-12)
    start_date = '2006-01-01'
    end_date = '2025-12-31'
    dates = pd.date_range(start=start_date, end=end_date, freq='MS') # 월의 시작일
    date_list = dates.strftime('%Y%m').tolist()
    
    print(f"총 요청 개수: {len(date_list)}")
    
    base_url = "https://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade"
    
    all_data = []
    
    create_directory('call_api')
    
    for i, ymd in enumerate(date_list):
        params = {
            'LAWD_CD': lawd_cd,
            'DEAL_YMD': ymd,
            'numOfRows': 10000,
            'serviceKey': service_key
        }
        
        try:
            response = requests.get(base_url, params=params)
            
            if response.status_code == 200:
                root = ET.fromstring(response.content)
                items = root.findall('.//item')
                
                for item in items:
                    def get_text(tag):
                        node = item.find(tag)
                        return node.text.strip() if node is not None else None

                    row = {
                        'year': get_text('년'),
                        'month': get_text('월'),
                        'day': get_text('일'),
                        'price': get_text('거래금액'),
                        'code': get_text('지역코드'),
                        'dong_nm': get_text('법정동'),
                        'jibun': get_text('지번'),
                        'con_year': get_text('건축년도'),
                        'apt_nm': get_text('아파트'),
                        'area': get_text('전용면적'),
                        'floor': get_text('층')
                    }
                    all_data.append(row)
            else:
                print(f"Error {response.status_code} at {ymd}")
                
        except Exception as e:
            print(f"Exception at {ymd}: {e}")
            
        if i % 10 == 0:
            print(f"진행상황: {i}/{len(date_list)}")
            
        time.sleep(0.1)

    df = pd.DataFrame(all_data)
    df.to_csv('apt_price2.csv', index=False, encoding='utf-8-sig')
    print(">>> 데이터 수집 완료 및 저장 (apt_price2.csv)")
    return df

## 2. 데이터 전처리 (Data Preprocessing)
가격 데이터를 숫자형으로 변환하고 평당 가격(py)을 계산합니다.

In [7]:
def preprocess_data():
    print(">>> 데이터 전처리 시작")
    if not os.path.exists('apt_price2.csv'):
        print("파일이 없어 수집 단계를 먼저 실행해야 합니다.")
        # collect_data() # 필요시 주석 해제
        return

    price_df = pd.read_csv('apt_price2.csv')
    
    if 'Unnamed: 0' in price_df.columns:
        price_df = price_df.drop(columns=['Unnamed: 0'])
    price_df = price_df.drop(columns=['code'], errors='ignore')

    # 결측값 확인
    print("결측값 수:\n", price_df.isnull().sum())

    # 가격, 면적 숫자 변환
    price_df['price'] = price_df['price'].astype(str).str.replace(',', '').str.strip()
    price_df['price'] = pd.to_numeric(price_df['price'], errors='coerce')
    price_df['area'] = pd.to_numeric(price_df['area'], errors='coerce')

    # 날짜 컬럼 생성
    price_df['ymd'] = pd.to_datetime(price_df[['year', 'month', 'day']])
    price_df['ym'] = price_df['ymd'].dt.to_period('M').dt.to_timestamp()

    # 주소 지번 만들기
    price_df['juso_jibun'] = price_df['dong_nm'].fillna('') + " " + \
                             price_df['jibun'].fillna('') + " " + \
                             price_df['apt_nm'].fillna('')
    price_df['juso_jibun'] = price_df['juso_jibun'].str.replace('  ', ' ').str.strip()

    # 평당 가격 (py)
    price_df['py'] = round(price_df['price'] / price_df['area'] * 3.3, 0)
    
    price_df['cnt'] = 1
    
    price_df.to_csv('price_last.csv', index=False, encoding='utf-8-sig')
    print(">>> 전처리 완료 (price_last.csv)")
    return price_df

## 3. 카카오 API로 지오코딩 (Geocoding)
주소를 좌표(위도, 경도)로 변환합니다. **API Key 입력이 필요합니다.**

In [10]:
def geocode_addresses():
    print(">>> 지오코딩 시작")
    if not os.path.exists('price_last.csv'): return
    price_df = pd.read_csv('price_last.csv')
    
    apt_juso = price_df[['juso_jibun']].drop_duplicates().dropna()
    
    # [중요] 여기에 카카오 API 키를 입력하세요
    kakao_key = '4f6c32c41ec2eea6d42afdc7430c769b'
    headers = {'Authorization': f'KakaoAK {kakao_key}'}
    
    add_list = []
    total = len(apt_juso)

    for idx, row in apt_juso.iterrows():
        query = row['juso_jibun']
        url = 'https://dapi.kakao.com/v2/local/search/address.json'
        
        try:
            resp = requests.get(url, headers=headers, params={'query': query})
            if resp.status_code == 200:
                data = resp.json()
                if data['documents']:
                    doc = data['documents'][0]
                    add_list.append({
                        'juso_jibun': query,
                        'coord_x': doc['x'],
                        'coord_y': doc['y']
                    })
            else:
                pass # 에러 처리 로직
        except Exception as e:
            print(f"Error for {query}: {e}")
            
        if len(add_list) % 100 == 0:
            print(f"진행률: {len(add_list)/total*100:.2f}%")
            
    juso_geocoding = pd.DataFrame(add_list)
    juso_geocoding['coord_x'] = pd.to_numeric(juso_geocoding['coord_x'])
    juso_geocoding['coord_y'] = pd.to_numeric(juso_geocoding['coord_y'])
    
    juso_geocoding.to_csv('juso_geocoding.csv', index=False, encoding='utf-8-sig')
    print(">>> 지오코딩 완료 (juso_geocoding.csv)")
    return juso_geocoding

## 4. 지오 데이터프레임 및 그리드 생성
가격 정보에 좌표를 결합하고, 500m x 500m 분석용 격자를 생성합니다.

In [14]:
def create_geo_data_and_grid():
    print(">>> 지오 데이터프레임 및 그리드 작업 시작")
    
    price_df = pd.read_csv('price_last.csv')
    juso_geocoding = pd.read_csv('juso_geocoding.csv')
    
    apt_price = pd.merge(price_df, juso_geocoding, on='juso_jibun', how='left')
    apt_price = apt_price.dropna(subset=['coord_x', 'coord_y'])
    
    geometry = [Point(xy) for xy in zip(apt_price['coord_x'], apt_price['coord_y'])]
    geo_apt_price = gpd.GeoDataFrame(apt_price, geometry=geometry, crs="EPSG:4326")
    
    geo_apt_price.to_file('geo_apt_price.geojson', driver='GeoJSON')
    
    # 시흥시 Shapefile (경로 확인 필요)
    shp_path = r'C:\Users\USER\Documents\시흥시집값분석\시흥시행정동\(B031)국가기본공간정보(경기도 시흥시)_NF_A_G01106/NF_A_G01106.shp'
    if not os.path.exists(shp_path):
        print(f"경고: 쉐이프파일 경로가 존재하지 않습니다: {shp_path}")
        return geo_apt_price, None

    shp1 = gpd.read_file(shp_path, encoding='cp949')
    shp1_5186 = shp1.to_crs(epsg=5186)
    
    # 그리드 생성 (500m)
    minx, miny, maxx, maxy = shp1_5186.total_bounds
    step = 500
    
    grid_cells = []
    for x in np.arange(minx, maxx, step):
        for y in np.arange(miny, maxy, step):
            grid_cells.append(box(x, y, x+step, y+step))
            
    grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=shp1_5186.crs)
    grid_filtered = gpd.overlay(grid, shp1_5186, how='intersection')
    
    grid_filtered.to_file('grid_siheung.geojson', driver='GeoJSON')
    
    return geo_apt_price, grid_filtered

## 5. 공간 분석 및 시각화 (KDE & Leaflet)
가격 상승 구간을 식별하고 커널 밀도 분석(KDE)을 수행하여 지도에 시각화합니다.

In [15]:
def analyze_and_visualize_kde():
    print(">>> 분석 및 시각화(KDE) 시작")
    
    if not os.path.exists('geo_apt_price.geojson') or not os.path.exists('grid_siheung.geojson'):
        print("데이터 파일이 없습니다.")
        return

    geo_apt_price = gpd.read_file('geo_apt_price.geojson')
    grid_filtered = gpd.read_file('grid_siheung.geojson')
    
    # Spatial Join을 위한 좌표계 변환
    grid_5186 = grid_filtered.to_crs(epsg=5186)
    geo_apt_5186 = geo_apt_price.to_crs(epsg=5186)
    
    joined = gpd.sjoin(geo_apt_5186, grid_5186, how='left', predicate='intersects')
    
    joined['ymd'] = pd.to_datetime(joined['ymd'])
    before = joined[joined['ymd'] <= '2020-12-31'].groupby('juso_jibun')['py'].mean().rename('before')
    after = joined[joined['ymd'] >= '2021-01-01'].groupby('juso_jibun')['py'].mean().rename('after')
    
    diff_df = pd.concat([before, after], axis=1)
    diff_df['diff'] = ((diff_df['after'] - diff_df['before']) / diff_df['before'] * 100).round(0)
    
    hot_df = diff_df[diff_df['diff'] > 0].dropna()
    
    coords = geo_apt_price.drop_duplicates('juso_jibun')[['juso_jibun', 'geometry']]
    hot_geo = pd.merge(hot_df, coords, on='juso_jibun', how='inner')
    hot_geo = gpd.GeoDataFrame(hot_geo, geometry='geometry', crs="EPSG:4326")
    
    if len(hot_geo) < 2:
        print("KDE 분석을 위한 데이터가 부족합니다.")
        return

    # Heatmap 시각화
    center = [hot_geo.geometry.y.mean(), hot_geo.geometry.x.mean()]
    m = folium.Map(location=center, zoom_start=12, tiles='cartodbpositron')
    
    heat_data = [[row.geometry.y, row.geometry.x, row['diff']] for idx, row in hot_geo.iterrows()]
    plugins.HeatMap(heat_data, radius=15, blur=20, max_zoom=1).add_to(m)
    
    m.save('kde_hot_map.html')
    print(">>> 시각화 완료 (kde_hot_map.html 저장됨)")

# 함수 실행 (필요한 단계의 주석을 해제하세요)
# collect_data()
# preprocess_data()
# geocode_addresses()
# create_geo_data_and_grid()
# analyze_and_visualize_kde()

In [16]:
# 함수 실행 (필요한 단계의 주석을 해제하세요)
collect_data()


>>> 데이터 수집 시작
총 요청 개수: 240
진행상황: 0/240
진행상황: 10/240
진행상황: 20/240
진행상황: 30/240
진행상황: 40/240
진행상황: 50/240
진행상황: 60/240
진행상황: 70/240
진행상황: 80/240
진행상황: 90/240
진행상황: 100/240
진행상황: 110/240
진행상황: 120/240
진행상황: 130/240
진행상황: 140/240
진행상황: 150/240
진행상황: 160/240
진행상황: 170/240
진행상황: 180/240
진행상황: 190/240
진행상황: 200/240
진행상황: 210/240
진행상황: 220/240
진행상황: 230/240
>>> 데이터 수집 완료 및 저장 (apt_price2.csv)


,year,month,day,price,code,dong_nm,jibun,con_year,apt_nm,area,floor
0,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
125327,None,None,None,None,None,None,None,None,None,None,None
125328,None,None,None,None,None,None,None,None,None,None,None
125329,None,None,None,None,None,None,None,None,None,None,None
125330,None,None,None,None,None,None,None,None,None,None,None
